In [21]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
%matplotlib inline

In [14]:
BASE_PATH = 'crop_img/img/CLOTHING/Blouse/'
ids = sorted([x for x in os.listdir(BASE_PATH)])

In [18]:
ids[:5]

['id_00004848', 'id_00005025', 'id_00005661', 'id_00005855', 'id_00006033']

In [24]:
for i in ids:
    files = sorted([x for x in os.listdir(BASE_PATH+i)])
    con = sorted([x for x in files if 'comsumer' in x])
    shop = sorted([x for x in files if 'shop' in x ])
    comb = list(itertools.product(tuple(con),tuple(shop)))
    print(comb)

[('comsumer_01.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg'), ('comsumer_02.jpg', 'shop_01.jpg'), ('comsumer_03.jpg', 'shop_01.jpg'), ('comsumer_04.jpg', 'shop_01.jpg'), ('comsumer_05.jpg', 'shop_01.jpg'), ('comsumer_06.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg'), ('comsumer_02.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg'), ('comsumer_01.jpg', 'shop_02.jpg'), ('comsumer_02.jpg', 'shop_01.jpg'), ('comsumer_02.jpg', 'shop_02.jpg'), ('comsumer_03.jpg', 'shop_01.jpg'), ('comsumer_03.jpg', 'shop_02.jpg'), ('comsumer_04.jpg', 'shop_01.jpg'), ('comsumer_04.jpg', 'shop_02.jpg'), ('comsumer_05.jpg', 'shop_01.jpg'), ('comsumer_05.jpg', 'shop_02.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg'), ('comsumer_02.jpg', 'shop_01.jpg'), ('comsumer_03.jpg', 'shop_01.jpg')]
[('comsumer_01.jpg', 'shop_01.jpg'), ('comsumer_01.jpg'

['comsumer_01.jpg', 'shop_01.jpg']